# CMIP5 build intake catalog

Build intake catalog for CMIP5 from manifest files.

In [1]:
import pandas as pd

## Load variable description

In [2]:
import yaml
import re

In [3]:
variables = dict()

variables_ = yaml.safe_load(open("../cmip5/monthly_single-level_variables.yml"))['variables']
variables.update(variables_)

variables_ = yaml.safe_load(open("../cmip5/monthly_pressure-levels_variables.yml"))['variables']
variables.update(variables_)

variables_ = yaml.safe_load(open("../cmip5/daily_single-level_variables.yml"))['variables']
variables.update(variables_)

variables_ = yaml.safe_load(open("../cmip5/daily_pressure_level_variables.yml"))['variables']
variables.update(variables_)

variables

{'Near-surface (2m) air temperature (tas)': {'units': 'K',
  'description': 'Temperature of the air near the surface.'},
 'Surface skin temperature (ts)': {'units': 'K',
  'description': 'Temperature at the interface (not the bulk temperature of the medium above or below) between air and sea for open-sea regions.'},
 'Daily maximum near-surface air temperature (tasmax)': {'units': 'K',
  'description': 'Daily maximum of the temperature of the air near the surface.'},
 'Daily minimum near-surface air temperature (tasmin)': {'units': 'K',
  'description': 'Daily minimum near-surface air temperature.'},
 'Sea-level pressure (psl)': {'units': 'Pa',
  'description': 'Pressure of the air pressure at sea level.'},
 'Surface pressure (ps)': {'units': 'Pa',
  'description': 'Pressure of air at the lower boundary of the atmopshere'},
 'Near-surface zonal component of wind (uas)': {'units': 'm.s^-1',
  'description': 'Magnitude of the zonal (eastward) component of the two-dimensional horizontal a

In [4]:
lookup = dict()
pattern = re.compile(r'\(([\w]+)\)')
for var in variables:
    matches = pattern.findall(var)
    if not matches:
        continue
    short_name = matches[-1]
    lookup[short_name] = dict()
    lookup[short_name]['variable_name'] = var
    lookup[short_name]['units'] = variables[var]['units']
    lookup[short_name]['description'] = variables[var]['description']
    
lookup

{'tas': {'variable_name': 'Near-surface (2m) air temperature (tas)',
  'units': 'K',
  'description': 'Temperature of the air near the surface.'},
 'ts': {'variable_name': 'Surface skin temperature (ts)',
  'units': 'K',
  'description': 'Temperature at the interface (not the bulk temperature of the medium above or below) between air and sea for open-sea regions.'},
 'tasmax': {'variable_name': 'Daily maximum near-surface air temperature (tasmax)',
  'units': 'K',
  'description': 'Daily maximum of the temperature of the air near the surface.'},
 'tasmin': {'variable_name': 'Daily mimimum near-surface air temperature (tasmin)',
  'units': 'K',
  'description': 'Daily minimum of the temperature of the air near the surface.'},
 'psl': {'variable_name': 'Mean sea level pressure (psl)',
  'units': 'Pa',
  'description': 'Air pressure at sea level.'},
 'ps': {'variable_name': 'Surface pressure (ps)',
  'units': 'Pa',
  'description': 'Pressure of air at the lower boundary of the atmopshere'

## Load manifest

In [5]:
df = pd.DataFrame(columns=["url", "label"])

df_m = pd.read_csv("../cmip5/manifest_C3S2_380_CMIP5_monthly_single_level_20230131.txt.gz", names=["url"])
df_m["label"] = "cmip5-monthly-single-level"
df = pd.concat([df, df_m], ignore_index=True)

# not in CDS
#df_m = pd.read_csv("../cmip5/manifest_C3S-34a-Lot1_cmip5-monthly-latitude-depth_http_latest.txt.gz", names=["url"])
#df_m["label"] = "cmip5-monthly-latitude-depth"
#df = pd.concat([df, df_m], ignore_index=True)

df_m = pd.read_csv("../cmip5/manifest_C3S2_380_CMIP5_monthly_pressure_level_20230131.txt.gz", names=["url"])
df_m["label"] = "cmip5-monthly-pressure-level"
df = pd.concat([df, df_m], ignore_index=True)

df_m = pd.read_csv("../cmip5/manifest_C3S2_380_CMIP5_daily_single_level_20230131.txt.gz", names=["url"])
df_m["label"] = "cmip5-daily-single-level"
df = pd.concat([df, df_m], ignore_index=True)

df_m = pd.read_csv("../cmip5/manifest_C3S2_380_CMIP5_daily_pressure_level_20230131.txt.gz", names=["url"])
df_m["label"] = "cmip5-daily-pressure-level"
df = pd.concat([df, df_m], ignore_index=True)

# not in CDS
#df_m = pd.read_csv("../cmip5/manifest_C3S-34a-Lot1_cmip5-3hr-single-level_http_latest.txt.gz", names=["url"])
#df_m["label"] = "cmip5-3hr-single-level"
#df = pd.concat([df, df_m], ignore_index=True)

df

,url,label
0,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
1,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
2,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
3,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
4,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
...,...,...
171682,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171683,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171684,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171685,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level


In [6]:
df.url[0]

'https://data.mips.copernicus-climate.eu/thredds/fileServer/esg_c3s-cmip5/output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i1p1/clt/v20181201/clt_Amon_bcc-csm1-1_amip_r1i1p1_197901-200812.nc'

## Build new dataframe


In [7]:
df_new = pd.DataFrame(columns = [
    'ds_id', 
    'path',
    'size',
    'project',
    'product',
    'institute',
    'model',
    'experiment',
    'time_frequency',
    'realm',
    'table',
    'ensemble',
    'variable',
    'version',
    'start_time',
    'end_time',
    'bbox',
    'level',
    'units',
    'variable_name',
    'description',
    'url',
    'label',
])
df_new

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label


In [8]:
df_new["path"] = df["url"].str.replace("https://data.mips.copernicus-climate.eu/thredds/fileServer/esg_c3s-cmip5/", "", regex=False)
df_new.head()

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,NaN,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# c3s-cmip5.output1.MPI-M.MPI-ESM-LR.historical.mon.atmos.Amon.r1i1p1.tas.v20120315

In [10]:
df_new["ds_id"] = df_new["path"].apply(lambda x: 'c3s-cmip5.' + '.'.join(x.split('/')[0:-1]))
df_new.head()

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_new["ds_id"][0].split(".")

['c3s-cmip5',
 'output1',
 'BCC',
 'bcc-csm1-1',
 'amip',
 'mon',
 'atmos',
 'Amon',
 'r1i1p1',
 'clt',
 'v20181201']

In [12]:
df_new['project'] = df_new["ds_id"].apply(lambda x: x.split('.')[0])
df_new['product'] = df_new["ds_id"].apply(lambda x: x.split('.')[1])
df_new['institute'] = df_new["ds_id"].apply(lambda x: x.split('.')[2])
df_new['model'] = df_new["ds_id"].apply(lambda x: x.split('.')[3])
df_new['experiment'] = df_new["ds_id"].apply(lambda x: x.split('.')[4])
df_new['time_frequency'] = df_new["ds_id"].apply(lambda x: x.split('.')[5])
df_new['realm'] = df_new["ds_id"].apply(lambda x: x.split('.')[6])
df_new['table'] = df_new["ds_id"].apply(lambda x: x.split('.')[7])
df_new['ensemble'] = df_new["ds_id"].apply(lambda x: x.split('.')[8])
df_new['variable'] = df_new["ds_id"].apply(lambda x: x.split('.')[9])
df_new['version'] = df_new["ds_id"].apply(lambda x: x.split('.')[10])
df_new['variable_name'] = df_new["variable"].apply(lambda x: lookup.get(x, {}).get('variable_name'))
df_new['units'] = df_new["variable"].apply(lambda x: lookup.get(x, {}).get('units'))
df_new['description'] = df_new["variable"].apply(lambda x: lookup.get(x, {}).get('description'))
df_new['url'] = df["url"].apply(lambda x: x)
df_new['label'] = df["label"].apply(lambda x: x)
df_new.head()

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,NaN,NaN,NaN,NaN,1,Total cloud fraction (clt),"Total refers to the whole atmosphere column, a...",https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,NaN,NaN,NaN,NaN,kg.m^-2.s^-1,Evaporation (evspsbl),Evaporation rate. It includes conversion to va...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,NaN,NaN,NaN,NaN,W.m^-2,Surface latent heat flux (hfls),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,NaN,NaN,NaN,NaN,W.m^-2,Surface sensible heat flux (hfss),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,NaN,NaN,NaN,NaN,kg.m-^2.s^-1,Snowfall flux (prsn),Mass of water in the form of snow precipitatin...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level


In [13]:
from datetime import datetime
import calendar

def parse_time(filename):
    time_part = filename.split("_")[-1].split(".nc")[0]
    if "-" not in time_part:
        return None, None
    start, end = time_part.split("-")
    if len(start) == 12:
        format = "%Y%m%d%H%M"
    elif len(start) == 8:
        format = "%Y%m%d"
    elif len(start) == 6:
        format = "%Y%m"
    start_time = datetime.strptime(start, format)
    if len(start) == 6:
        start_time = datetime(start_time.year, start_time.month, 1, 12)
    elif len(start) == 8:
        start_time = datetime(start_time.year, start_time.month, start_time.day, 12)
    end_time = datetime.strptime(end, format)
    if len(end) == 6:
        end_time = datetime(end_time.year, end_time.month, calendar.monthrange(end_time.year, end_time.month)[1], 12)
    elif len(end) == 8:
        end_time = datetime(end_time.year, end_time.month, end_time.day, 12)
    return start_time.isoformat(), end_time.isoformat()

In [14]:
df_new["path"][0].split("/")[-1]

'clt_Amon_bcc-csm1-1_amip_r1i1p1_197901-200812.nc'

In [15]:
df_new['start_time'] = df_new["path"].apply(lambda x: parse_time(x.split('/')[-1])[0])
df_new['end_time'] = df_new["path"].apply(lambda x: parse_time(x.split('/')[-1])[1])
df_new.head()

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,1,Total cloud fraction (clt),"Total refers to the whole atmosphere column, a...",https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m^-2.s^-1,Evaporation (evspsbl),Evaporation rate. It includes conversion to va...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface latent heat flux (hfls),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface sensible heat flux (hfss),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m-^2.s^-1,Snowfall flux (prsn),Mass of water in the form of snow precipitatin...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level


## Write catalog

In [16]:
last_updated = datetime.now().utcnow()
version = last_updated.strftime('v%Y%m%d')
cat_name = f"c3s-cmip5_{version}.csv.gz"
cat_path = f"../intake/catalogs/c3s-cmip5/{cat_name}"

df_new.to_csv(cat_path, index=False, compression="gzip")

### Write individual catalogs for CDS

In [17]:
labels = list(df_new.label.unique())
labels

['cmip5-monthly-single-level',
 'cmip5-monthly-pressure-level',
 'cmip5-daily-single-level',
 'cmip5-daily-pressure-level']

In [18]:
for label in labels:
    cat_subset_name = f"c3s-{label}_{version}.csv.gz"
    cat_subset_path = f"../intake/catalogs/c3s-cmip5/{cat_subset_name}"
    df_subset = df_new[df_new.label==label]
    print(df_subset.nunique())
    df_subset.to_csv(cat_subset_path, index=False, compression="gzip")

ds_id             19418
path              96754
size                  0
project               1
product               1
institute            20
model                46
experiment            7
time_frequency        1
realm                 5
table                 5
ensemble             40
variable             40
version             232
start_time          535
end_time            545
bbox                  0
level                 0
units                13
variable_name        39
description          39
url               96754
label                 1
dtype: int64
ds_id              2530
path              18713
size                  0
project               1
product               1
institute            19
model                44
experiment            7
time_frequency        1
realm                 1
table                 1
ensemble             36
variable              6
version             139
start_time          456
end_time            462
bbox                  0
level                 0
uni

 ## Load catalog

In [19]:
df_c = pd.read_csv(cat_path)
df_c

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,1,Total cloud fraction (clt),"Total refers to the whole atmosphere column, a...",https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m^-2.s^-1,Evaporation (evspsbl),Evaporation rate. It includes conversion to va...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface latent heat flux (hfls),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface sensible heat flux (hfss),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m-^2.s^-1,Snowfall flux (prsn),Mass of water in the form of snow precipitatin...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171682,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2004-01-01T12:00:00,2004-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171683,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2005-01-01T12:00:00,2005-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171684,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2006-01-01T12:00:00,2006-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171685,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2007-01-01T12:00:00,2007-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level


In [20]:
df_c.nunique()

ds_id              26034
path              171687
size                   0
project                1
product                1
institute             20
model                 46
experiment             7
time_frequency         2
realm                  5
table                  6
ensemble              40
variable              47
version              263
start_time          1077
end_time            1120
bbox                   0
level                  0
units                 15
variable_name         46
description           46
url               171687
label                  4
dtype: int64

## Load intake

In [21]:
import intake

In [22]:
cat_url = "https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml"
# cat_url = "https://github.com/cp4cds/c3s_34g_manifests/raw/update-cmip5-20230131/intake/catalogs/c3s.yaml"
cat = intake.open_catalog(cat_url)
list(cat)

['c3s-cmip5',
 'c3s-cmip5-daily-pressure-level',
 'c3s-cmip5-daily-single-level',
 'c3s-cmip5-monthly-pressure-level',
 'c3s-cmip5-monthly-single-level',
 'c3s-cmip6',
 'c3s-cmip6-r3',
 'c3s-cmip6-decadal',
 'c3s-cmip6-decadal-dwd',
 'c3s-cordex',
 'c3s-ipcc-atlas']

In [23]:
df_cmip5 = cat['c3s-cmip5'].read()
df_cmip5

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,1,Total cloud fraction (clt),"Total refers to the whole atmosphere column, a...",https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m^-2.s^-1,Evaporation (evspsbl),Evaporation rate. It includes conversion to va...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface latent heat flux (hfls),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface sensible heat flux (hfss),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m-^2.s^-1,Snowfall flux (prsn),Mass of water in the form of snow precipitatin...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171682,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2004-01-01T12:00:00,2004-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171683,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2005-01-01T12:00:00,2005-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171684,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2006-01-01T12:00:00,2006-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171685,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2007-01-01T12:00:00,2007-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level


In [24]:
df_cmip5.nunique()

ds_id              26034
path              171687
size                   0
project                1
product                1
institute             20
model                 46
experiment             7
time_frequency         2
realm                  5
table                  6
ensemble              40
variable              47
version              263
start_time          1077
end_time            1120
bbox                   0
level                  0
units                 15
variable_name         46
description           46
url               171687
label                  4
dtype: int64

### Subset intake catalog

In [25]:
df_cmip5_subset = cat['c3s-cmip5-daily-pressure-level'].read()
df_cmip5_subset

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v1,1979-01-01T12:00:00,1993-12-31T12:00:00,NaN,NaN,m s-1,U-component of wind (ua),Magnitude of the eastward component of the two...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v1,1994-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,m s-1,U-component of wind (ua),Magnitude of the eastward component of the two...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v20181201,1979-01-01T12:00:00,1993-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v20181201,1994-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.historical.da...,output1/BCC/bcc-csm1-1/historical/day/atmos/da...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,historical,day,atmos,...,v20181201,1950-01-01T12:00:00,1965-12-31T12:00:00,NaN,NaN,K,Air temperature (ta),Temperature of the air.,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17399,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2004-01-01T12:00:00,2004-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
17400,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2005-01-01T12:00:00,2005-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
17401,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2006-01-01T12:00:00,2006-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
17402,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2007-01-01T12:00:00,2007-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level


In [26]:
df_cmip5_subset = cat['c3s-cmip5-daily-single-level'].read()
df_cmip5_subset

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m-^2.s^-1,Snowfall flux (prsn),Mass of water in the form of snow precipitatin...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v1,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m^-2.s^-1,Mean precipitation flux (pr),Amount of water per unit area and time.,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v1,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,Pa,Mean sea level pressure (psl),Air pressure at sea level.,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Near-surface downwelling shortwave radiation(r...,Radiative shortwave flux of energy downward at...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.day.atmo...,output1/BCC/bcc-csm1-1/amip/day/atmos/day/r1i1...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,day,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,K,Daily maximum near-surface air temperature (ta...,Daily maximum of the temperature of the air ne...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38811,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-CAM5.rcp8...,output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/day/atmo...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-CAM5,rcp85,day,atmos,...,v20130709,2041-01-01T12:00:00,2075-12-31T12:00:00,NaN,NaN,K,Daily mimimum near-surface air temperature (ta...,Daily minimum of the temperature of the air ne...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level
38812,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-CAM5.rcp8...,output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/day/atmo...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-CAM5,rcp85,day,atmos,...,v20130709,2076-01-01T12:00:00,2100-12-31T12:00:00,NaN,NaN,K,Daily mimimum near-surface air temperature (ta...,Daily minimum of the temperature of the air ne...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level
38813,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-CAM5.rcp8...,output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/day/atmo...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-CAM5,rcp85,day,atmos,...,v20130709,2006-01-01T12:00:00,2040-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level
38814,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-CAM5.rcp8...,output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/day/atmo...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-CAM5,rcp85,day,atmos,...,v20130709,2041-01-01T12:00:00,2075-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-single-level


In [27]:
df_cmip5_subset = cat['c3s-cmip5-monthly-pressure-level'].read()
df_cmip5_subset

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,%,Relative humidity (hur),Amount of moisture in the air divided by the m...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,fraction,Specific humidity (hus),Amount of moisture in the air divided by amoun...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,K,Air temperature (ta),Temperature of the air.,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,m s-1,U-component of wind (ua),Magnitude of the eastward component of the two...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v1,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,m.s^-1,Meridional component of Wind (va),Magnitude of the meridional (northward) compon...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18708,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-WACCM.rcp...,output1/NSF-DOE-NCAR/CESM1-WACCM/rcp85/mon/atm...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-WACCM,rcp85,mon,atmos,...,v20130315,2051-01-01T12:00:00,2099-12-31T12:00:00,NaN,NaN,m s-1,U-component of wind (ua),Magnitude of the eastward component of the two...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level
18709,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-WACCM.rcp...,output1/NSF-DOE-NCAR/CESM1-WACCM/rcp85/mon/atm...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-WACCM,rcp85,mon,atmos,...,v20130315,2006-01-01T12:00:00,2050-12-31T12:00:00,NaN,NaN,m.s^-1,Meridional component of Wind (va),Magnitude of the meridional (northward) compon...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level
18710,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-WACCM.rcp...,output1/NSF-DOE-NCAR/CESM1-WACCM/rcp85/mon/atm...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-WACCM,rcp85,mon,atmos,...,v20130315,2051-01-01T12:00:00,2099-12-31T12:00:00,NaN,NaN,m.s^-1,Meridional component of Wind (va),Magnitude of the meridional (northward) compon...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level
18711,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-WACCM.rcp...,output1/NSF-DOE-NCAR/CESM1-WACCM/rcp85/mon/atm...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-WACCM,rcp85,mon,atmos,...,v20130315,2006-01-01T12:00:00,2050-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-pressure-level


In [28]:
df_cmip5_subset = cat['c3s-cmip5-monthly-single-level'].read()
df_cmip5_subset

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,1,Total cloud fraction (clt),"Total refers to the whole atmosphere column, a...",https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m^-2.s^-1,Evaporation (evspsbl),Evaporation rate. It includes conversion to va...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface latent heat flux (hfls),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface sensible heat flux (hfss),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m-^2.s^-1,Snowfall flux (prsn),Mass of water in the form of snow precipitatin...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96749,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-WACCM.rcp...,output1/NSF-DOE-NCAR/CESM1-WACCM/rcp85/mon/lan...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-WACCM,rcp85,mon,land,...,v20140403,2056-01-01T12:00:00,2099-12-31T12:00:00,NaN,NaN,kg.m^-2,Total column soil moisture content (mrsos),Vertical sum per unit area from the surface do...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
96750,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-WACCM.rcp...,output1/NSF-DOE-NCAR/CESM1-WACCM/rcp85/mon/lan...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-WACCM,rcp85,mon,land,...,v20140403,2006-01-01T12:00:00,2050-12-31T12:00:00,NaN,NaN,kg.m^-2.s^-1,Total runoff (mrro),Amount per unit area of surface and subsurface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
96751,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-WACCM.rcp...,output1/NSF-DOE-NCAR/CESM1-WACCM/rcp85/mon/lan...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-WACCM,rcp85,mon,land,...,v20140403,2051-01-01T12:00:00,2099-12-31T12:00:00,NaN,NaN,kg.m^-2.s^-1,Total runoff (mrro),Amount per unit area of surface and subsurface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
96752,c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-WACCM.rcp...,output1/NSF-DOE-NCAR/CESM1-WACCM/rcp85/mon/lan...,NaN,c3s-cmip5,output1,NSF-DOE-NCAR,CESM1-WACCM,rcp85,mon,land,...,v20140403,2006-01-01T12:00:00,2050-12-31T12:00:00,NaN,NaN,kg.m^-2,Total column soil moisture content (mrsos),Vertical sum per unit area from the surface do...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
